In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

In [2]:
from pyalm.models.openai import OpenAI
import os

llm = OpenAI("gpt-4o-2024-08-06", os.getenv("OPENAI_API_KEY"), azure_endpoint=os.getenv("AZURE_ENDPOINT"), n_ctx = 15000)

In [8]:
from rixeval.runner import *

# from rixeval import prompts
# new_prompts = {key: value[:8] for key, value in prompts.prompts.items()}

from rixeval import prompt_pieces
prompt_types = {key: value[:8] for key, value in prompt_pieces.prompts.items()}
direct_answers = prompt_pieces.direct_answers.direct_answers[:8]
cot = prompt_pieces.cot.cot[:8]

print(prompt_types.keys())

# for i in ['ai_trust_emotional', 'ai_distrust_emotional', 'ai_trust_rational', 'ai_distrust_rational']:
#     del prompt_types[i]
# print(prompt_types.keys())

avoid_fn = prompt_pieces.bias.avoid_fn[:1]
avoid_fp = prompt_pieces.bias.avoid_fp[:1]

variations = {
    "final_prompt_type": NamedCategories("final_prompt_type", prompt_types),
    # "final_prompt_type2": NamedCategories("final_prompt_type2", prompt_types),
    "think_mode": NamedCategories("think_mode", {"direct": direct_answers, "cot": cot}),
    # "xai_excluded" : CategoricalList("xai_excluded", ["yes", "no"])
    # "bias" : NamedCategories("bias", {"avoid_fn": avoid_fn, "avoid_fp": avoid_fp, "None": [None]}),
    # "final_prompt_type": NamedCategories("final_prompt_type", new_prompts),
    # "confidence_score": SequentialRange("confidence_score", 50, 99, 6, True),
    # "accuracy": UniformRange("accuracy", 60, 100, 2),
    # "accuracy": SequentialRange("accuracy", 50, 95, 3, True),
    # "xai_exclusion": Exclusion("xai_methods", ["shap", "pdp", "lime_explanation", "counterfactual1", "histograms"]),
    # "method_order": Permutation("method_order", ["shap", "lime_explanation", "pdp", "histogram"], max_perms=5)
}


from rixeval import dataload
xaidata = dataload.XAIData()

# Run experiment
config = ExperimentConfig("temp", "prompt and confidence", variations, max_datapoints=30, verbose=True, no_llm=True,
                          )
#static_parameters= {"accuracy":"unknown"})
#{"confidence":None}
#{'xai_exclusion': {'shap': True,'pdp': True,'lime_explanation': True,'counterfactual1': True, 'histograms': True}}
ExperimentRunner.get_runstats(config)
runner = ExperimentRunner(config, xaidata, llm)
# results_df = runner.run()

dict_keys(['ai_trust_emotional', 'ai_distrust_emotional', 'ai_neutral_emotional', 'ai_trust_rational', 'ai_distrust_rational', 'ai_neutral_rational'])
Total combinations: 768
Total expected runs: 23040
This will take between 691-1114 minutes / 11.52-18.56 hours.


In [14]:
results_df

,experiment_id,combination_id,datapoint_idx,dataset_index,confidence,true_outcome,predicted_outcome,llm_decision,reasoning_length,processing_time,timestamp,accuracy,xai_exclusion_shap,xai_exclusion_pdp
0,test_experiment,0,0,8723,100.00,Check-Out,Check-Out,Cancellation,0,0.010108,1.748455e+09,90.429235,False,False
1,test_experiment,0,1,10233,72.29,Check-Out,Check-Out,Cancellation,0,0.010105,1.748455e+09,90.429235,False,False
2,test_experiment,0,2,52028,79.40,Cancellation,Cancellation,Cancellation,0,0.010121,1.748455e+09,90.429235,False,False
3,test_experiment,0,3,16948,81.32,Check-Out,Cancellation,Cancellation,0,0.010121,1.748455e+09,90.429235,False,False
4,test_experiment,0,4,16376,100.00,Check-Out,Check-Out,Cancellation,0,0.010154,1.748455e+09,90.429235,False,False
5,test_experiment,1,0,8723,100.00,Check-Out,Check-Out,Cancellation,0,0.010098,1.748455e+09,90.429235,True,False
6,test_experiment,1,1,10233,72.29,Check-Out,Check-Out,Cancellation,0,0.010103,1.748455e+09,90.429235,True,False
7,test_experiment,1,2,52028,79.40,Cancellation,Cancellation,Cancellation,0,0.010097,1.748455e+09,90.429235,True,False
8,test_experiment,1,3,16948,81.32,Check-Out,Cancellation,Cancellation,0,0.010112,1.748455e+09,90.429235,True,False
9,test_experiment,1,4,16376,100.00,Check-Out,Check-Out,Cancellation,0,0.010118,1.748455e+09,90.429235,True,False


In [63]:
from rixeval import dataload
data = dataload.XAIData()

In [76]:
from rixeval import prompt_builder
pb = prompt_builder.PromptBuilder(data)
pb.print_outcome(4)


Predicted target: Check-Out, Actual target: Check-Out


In [98]:
from rixeval import prompt_builder
pb = prompt_builder.PromptBuilder(data)

llm_decisions = pd.DataFrame(columns=["index", "confidence", "true_outcome", "predicted_outcome", "llm_decision"])

for index in range(len(data.datapoints)):
    print(f"Doing index {index}")
    msg = pb.build_full_prompt(index)
    answer = llm.create_completion(msg)
    if answer.endswith("#CANCELLATION#"):
        decision = "Cancellation"
    elif answer.endswith("#CHECK-OUT#"):
        decision = "Check-Out"
    else:
        decision = "INVALID"
    true_outcome = data.datapoints[index]["actual_target"]
    predicted_outcome = data.datapoints[index]["datapoint"]["predicted_target"]
    confidence = data.datapoints[index]["datapoint"]["confidence_score"]
    dataset_index = data.datapoints[index]["datapoint_id"]
    print(f"Model decision for index {index} with true outcome {true_outcome}: {decision}")
    llm_decisions.loc[index] = [dataset_index, confidence, true_outcome, predicted_outcome, decision]

llm_decisions


Doing index 0
Model decision for index 0 with true outcome Check-Out: Check-Out
Doing index 1
Model decision for index 1 with true outcome Check-Out: Check-Out
Doing index 2
Model decision for index 2 with true outcome Cancellation: Cancellation
Doing index 3
Model decision for index 3 with true outcome Check-Out: Cancellation
Doing index 4
Model decision for index 4 with true outcome Check-Out: Check-Out
Doing index 5
Model decision for index 5 with true outcome Cancellation: Cancellation
Doing index 6
Model decision for index 6 with true outcome Check-Out: Cancellation
Doing index 7
Model decision for index 7 with true outcome Check-Out: Check-Out
Doing index 8
Model decision for index 8 with true outcome Check-Out: Check-Out
Doing index 9
Model decision for index 9 with true outcome Check-Out: Check-Out
Doing index 10
Model decision for index 10 with true outcome Check-Out: Check-Out
Doing index 11
Model decision for index 11 with true outcome Cancellation: Check-Out
Doing index 12


KeyboardInterrupt: 

In [101]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from scipy.stats import pearsonr

# Assuming llm_decisions DataFrame already exists
# Let's encode the categorical variables numerically
# We'll use 1 for "Check-Out" and 0 for "Cancellation"

def encode_outcome(series):
    """Encode Check-Out as 1, Cancellation as 0"""
    return series.map({'Check-Out': 1, 'Cancellation': 0})

# Create encoded versions
llm_decisions['true_outcome_encoded'] = encode_outcome(llm_decisions['true_outcome'])
llm_decisions['predicted_outcome_encoded'] = encode_outcome(llm_decisions['predicted_outcome'])
llm_decisions['llm_decision_encoded'] = encode_outcome(llm_decisions['llm_decision'])

# Calculate correlations
corr_true_llm = np.corrcoef(llm_decisions['true_outcome_encoded'],
                           llm_decisions['llm_decision_encoded'])[0, 1]

corr_predicted_llm = np.corrcoef(llm_decisions['predicted_outcome_encoded'],
                                llm_decisions['llm_decision_encoded'])[0, 1]

# Alternative: using pandas corr() method
corr_true_llm_pandas = llm_decisions['true_outcome_encoded'].corr(llm_decisions['llm_decision_encoded'])
corr_predicted_llm_pandas = llm_decisions['predicted_outcome_encoded'].corr(llm_decisions['llm_decision_encoded'])

# Calculate Pearson correlation with p-values
corr_true_llm_pearson, p_value_true = pearsonr(llm_decisions['true_outcome_encoded'],
                                              llm_decisions['llm_decision_encoded'])
corr_predicted_llm_pearson, p_value_predicted = pearsonr(llm_decisions['predicted_outcome_encoded'],
                                                         llm_decisions['llm_decision_encoded'])

# Print results
print("Correlation Analysis Results:")
print("=" * 50)
print(f"True Outcome vs LLM Decision correlation: {corr_true_llm:.4f}")
print(f"Predicted Outcome vs LLM Decision correlation: {corr_predicted_llm:.4f}")
print()
print("With p-values:")
print(f"True Outcome vs LLM Decision: r = {corr_true_llm_pearson:.4f}, p = {p_value_true:.4f}")
print(f"Predicted Outcome vs LLM Decision: r = {corr_predicted_llm_pearson:.4f}, p = {p_value_predicted:.4f}")
print()

# Determine which is more correlated
if abs(corr_true_llm) > abs(corr_predicted_llm):
    print(f"TRUE OUTCOME is more correlated with LLM Decision")
    print(f"Difference: {abs(corr_true_llm) - abs(corr_predicted_llm):.4f}")
else:
    print(f"PREDICTED OUTCOME is more correlated with LLM Decision")
    print(f"Difference: {abs(corr_predicted_llm) - abs(corr_true_llm):.4f}")

# Let's also look at agreement rates (accuracy)
true_agreement = (llm_decisions['true_outcome'] == llm_decisions['llm_decision']).mean()
predicted_agreement = (llm_decisions['predicted_outcome'] == llm_decisions['llm_decision']).mean()

print()
print("Agreement Rates:")
print("=" * 50)
print(f"True Outcome vs LLM Decision agreement: {true_agreement:.4f} ({true_agreement*100:.2f}%)")
print(f"Predicted Outcome vs LLM Decision agreement: {predicted_agreement:.4f} ({predicted_agreement*100:.2f}%)")

# Create a summary table
summary_data = {
    'Comparison': ['True vs LLM', 'Predicted vs LLM'],
    'Correlation': [corr_true_llm, corr_predicted_llm],
    'P-value': [p_value_true, p_value_predicted],
    'Agreement Rate': [true_agreement, predicted_agreement]
}

summary_df = pd.DataFrame(summary_data)
print()
print("Summary Table:")
print("=" * 50)
print(summary_df.round(4))

# Optional: Create a confusion matrix-style analysis
print()
print("Detailed Breakdown:")
print("=" * 50)

# Cross-tabulation for true vs llm
print("True Outcome vs LLM Decision:")
true_crosstab = pd.crosstab(llm_decisions['true_outcome'],
                           llm_decisions['llm_decision'],
                           margins=True)
print(true_crosstab)

print()
print("Predicted Outcome vs LLM Decision:")
predicted_crosstab = pd.crosstab(llm_decisions['predicted_outcome'],
                                llm_decisions['llm_decision'],
                                margins=True)
print(predicted_crosstab)

Correlation Analysis Results:
True Outcome vs LLM Decision correlation: 0.2335
Predicted Outcome vs LLM Decision correlation: 0.8563

With p-values:
True Outcome vs LLM Decision: r = 0.2335, p = 0.3840
Predicted Outcome vs LLM Decision: r = 0.8563, p = 0.0000

PREDICTED OUTCOME is more correlated with LLM Decision
Difference: 0.6228

Agreement Rates:
True Outcome vs LLM Decision agreement: 0.6875 (68.75%)
Predicted Outcome vs LLM Decision agreement: 0.9375 (93.75%)

Summary Table:
         Comparison  Correlation  P-value  Agreement Rate
0       True vs LLM       0.2335    0.384          0.6875
1  Predicted vs LLM       0.8563    0.000          0.9375

Detailed Breakdown:
True Outcome vs LLM Decision:
llm_decision  Cancellation  Check-Out  All
true_outcome                              
Cancellation             2          3    5
Check-Out                2          9   11
All                      4         12   16

Predicted Outcome vs LLM Decision:
llm_decision       Cancellation  Check